In [1]:
import sys
import os

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.abspath('')
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
import re

import importlib
import utils
importlib.reload(utils)
from tqdm import tqdm
import pickle

from utils import get_parsed, get_parsed_remove_comment, baseball_base_url

In [3]:
content = get_parsed_remove_comment(baseball_base_url + '/leagues/')

In [4]:
leagues_active = content.find('table', id='leagues_active')
all_rows = leagues_active.find_all('tr')

Yearly result

In [5]:
columns_name = [th.text for th in all_rows[0].find_all('th')]
columns_name += [name + '_url' for name in columns_name[1:]]
columns_name

['Year', 'Lg', 'Champion', 'MVP', 'Lg_url', 'Champion_url', 'MVP_url']

In [6]:
data = []
for row in all_rows[1:]:
    year = row.find('th').attrs['csk']
    print(year)
    if int(year) < 1980:
        break
    values = [th.text.strip() for th in row.find_all('td')]
    values_url = [th.a['href'] if th.find('a') else '' for th in row.find_all('td')]
    print(values)
    print(values_url)
    data.append([year] + values + values_url)
year_league_df = pd.DataFrame(data, columns=columns_name)

2020
['NL', 'Los Angeles Dodgers', 'Freddie Freeman']
['/leagues/NL/2020.shtml', '/teams/LAD/2020.shtml', '/players/f/freemfr01.shtml']
2020
['AL', 'Tampa Bay Rays', 'José Abreu']
['/leagues/AL/2020.shtml', '/teams/TBR/2020.shtml', '/players/a/abreujo02.shtml']
2019
['NL', 'Washington Nationals', 'Cody Bellinger']
['/leagues/NL/2019.shtml', '/teams/WSN/2019.shtml', '/players/b/bellico01.shtml']
2019
['AL', 'Houston Astros', 'Mike Trout']
['/leagues/AL/2019.shtml', '/teams/HOU/2019.shtml', '/players/t/troutmi01.shtml']
2018
['NL', 'Los Angeles Dodgers', 'Christian Yelich']
['/leagues/NL/2018.shtml', '/teams/LAD/2018.shtml', '/players/y/yelicch01.shtml']
2018
['AL', 'Boston Red Sox', 'Mookie Betts']
['/leagues/AL/2018.shtml', '/teams/BOS/2018.shtml', '/players/b/bettsmo01.shtml']
2017
['NL', 'Los Angeles Dodgers', 'Giancarlo Stanton']
['/leagues/NL/2017.shtml', '/teams/LAD/2017.shtml', '/players/s/stantmi03.shtml']
2017
['AL', 'Houston Astros', 'Jose Altuve']
['/leagues/AL/2017.shtml', '

In [7]:
year_league_df.head(5)

,Year,Lg,Champion,MVP,Lg_url,Champion_url,MVP_url
0,2020,NL,Los Angeles Dodgers,Freddie Freeman,/leagues/NL/2020.shtml,/teams/LAD/2020.shtml,/players/f/freemfr01.shtml
1,2020,AL,Tampa Bay Rays,José Abreu,/leagues/AL/2020.shtml,/teams/TBR/2020.shtml,/players/a/abreujo02.shtml
2,2019,NL,Washington Nationals,Cody Bellinger,/leagues/NL/2019.shtml,/teams/WSN/2019.shtml,/players/b/bellico01.shtml
3,2019,AL,Houston Astros,Mike Trout,/leagues/AL/2019.shtml,/teams/HOU/2019.shtml,/players/t/troutmi01.shtml
4,2018,NL,Los Angeles Dodgers,Christian Yelich,/leagues/NL/2018.shtml,/teams/LAD/2018.shtml,/players/y/yelicch01.shtml


# Generate High Level Questions

In [24]:
wrong_count = 4
columns = ['Question', 'Correct'] + ['Wrong_' + str(i + 1) for i in range(wrong_count)] +  ['tags']
years_range = range(1980, 2021)


question_list = []
import random
random.seed(1000)
for i in range(year_league_df.shape[0]):
    year = year_league_df.loc[i, 'Year']
    if int(year) < 1980:
        break
    LN = 'National League' if year_league_df.loc[i, 'Lg'] == 'NL' else 'American League'
    
    question = f'Which team is the champion of {LN} in {year}?'
    correct = year_league_df.loc[i, 'Champion']
    options = year_league_df.loc[year_league_df['Champion'] != correct]['Champion'].tolist()
    wrongs = random.sample(options, k = wrong_count + 1)
    wrongs = [wrong for wrong in wrongs if wrong]
    wrongs = wrongs[:wrong_count]
    
    tags = ['Champions']
    question_list.append([question] + [correct] + wrongs + [tags])
    
    question = f'Which team is the MVP of {LN} in {year}?'
    correct = year_league_df.loc[i, 'MVP']
    options = year_league_df.loc[year_league_df['MVP'] != correct]['MVP'].tolist()
    wrongs = random.sample(options, k = wrong_count)
    
    tags = ['MVP']
    question_list.append([question] + [correct] + wrongs + [tags])

In [25]:
len(question_list)

164

In [26]:
league_chm_mvp_questions = pd.DataFrame(question_list, columns=columns)
with open('questions_league_champ_mvp.pkl', 'wb') as f:
    pickle.dump(league_chm_mvp_questions, f)
with open('questions_league_champ_mvp.pkl', 'rb') as f:
    league_chm_mvp_questions = pickle.load(f)

In [27]:
league_chm_mvp_questions

,Question,Correct,Wrong_1,Wrong_2,Wrong_3,Wrong_4,tags
0,Which team is the champion of National League ...,Los Angeles Dodgers,Toronto Blue Jays,Boston Red Sox,Atlanta Braves,Kansas City Royals,[Champions]
1,Which team is the MVP of National League in 2020?,Freddie Freeman,Barry Bonds,Albert Pujols,Roger Clemens,Barry Bonds,[MVP]
2,Which team is the champion of American League ...,Tampa Bay Rays,Detroit Tigers,Houston Astros,St. Louis Cardinals,Atlanta Braves,[Champions]
3,Which team is the MVP of American League in 2020?,José Abreu,Jimmy Rollins,Ken Griffey Jr.,Albert Pujols,Cal Ripken Jr.,[MVP]
4,Which team is the champion of National League ...,Washington Nationals,Philadelphia Phillies,Los Angeles Dodgers,Oakland Athletics,Texas Rangers,[Champions]
...,...,...,...,...,...,...,...
159,Which team is the MVP of American League in 1981?,Rollie Fingers,Cody Bellinger,Jose Canseco,Andre Dawson,Mike Schmidt,[MVP]
160,Which team is the champion of National League ...,Philadelphia Phillies,Tampa Bay Rays,Houston Astros,Kansas City Royals,Oakland Athletics,[Champions]
161,Which team is the MVP of National League in 1980?,Mike Schmidt,Rollie Fingers,Barry Bonds,Cal Ripken Jr.,Kirk Gibson,[MVP]
162,Which team is the champion of American League ...,Kansas City Royals,Detroit Tigers,Florida Marlins,Boston Red Sox,San Francisco Giants,[Champions]


In [8]:
folder = 'major_leagues'
year_league_df.to_csv(os.path.join(folder, 'major_leagues.csv'))

In [9]:
with open('pkl_year_league.pkl', 'wb') as f:
    pickle.dump(year_league_df, f)
with open('pkl_year_league.pkl', 'rb') as f:
    year_league_df = pickle.load(f)

## Scrape data for each year

In [20]:
folder = 'major_leagues'
def process_division(content):
    # for division
    division = content.find('div', attrs={'id':'div_standings'})
    divisions = division.div.find_all('table')
    #print(len(divisions))
    
    division_data = []
    for division in divisions:
        heading = division.parent.parent.previous_sibling.previous_sibling
        division_name = heading.text.strip()
        
        thead = division.thead
        division_columns = [th.text for th in thead.find_all('th')]
        division_columns = ['Division'] + division_columns
        
        tbody = division.tbody
        for tr in tbody.find_all('tr'):
            team = tr.find('th').text
            temp = [td.text for td in tr.find_all('td')]
            division_data.append([division_name, team] + temp)
    
    df_division = pd.DataFrame(division_data, columns=division_columns)
    return df_division

def process_post(content):
    post = content.find('div', attrs={'id':'all_postseason'})
    if not post:
        return pd.DataFrame([])
    
    table = post.find('table')
    
    data = []
    for tr in table.tbody.find_all('tr'):
        temp = [td.text.strip() for td in tr.find_all('td')]
        data.append(temp)
    
    df = pd.DataFrame(data, columns=['Series', 'Score', 'Over'])
    return df

def process_batting(content):
    batting = content.find('div', attrs={'id':'all_teams_standard_batting'})
    table = batting.find('table')
    head = table.thead.find('tr')
    columns_name = [th.text.strip() for th in head.find_all('th')]
    #print(columns_name)
    #print(len(columns_name))
    columns_exp_stat = [th.attrs['data-stat'] if 'data-stat' in th.attrs else '' for th in head.find_all('th')]
    columns_exp_tip = [th.attrs['data-tip']  if 'data-tip' in th.attrs else '' for th in head.find_all('th')]
    body = table.tbody
    data = []
    for tr in body.find_all('tr'):
        if 'class' not in tr.attrs:
            name_div = tr.find('th')
            team_name = name_div.a['title']
            temp = [td.text.strip() for td in tr.find_all('td')]
            data.append([team_name+'(' + name_div.text + ')'] + temp)
    
    df_batting = pd.DataFrame(data, columns=columns_name)
    df_batting_exp = pd.DataFrame([columns_exp_stat, columns_exp_tip], columns=columns_name)
    return df_batting, df_batting_exp

def process_pitching(content):
    pitching = content.find('div', attrs={'id':'all_teams_standard_pitching'})
    table = pitching.find('table')
    head = table.thead.find('tr')
    columns_name = [th.text.strip() for th in head.find_all('th')]
    #print(columns_name)
    #print(len(columns_name))
    columns_exp_stat = [th.attrs['data-stat'] if 'data-stat' in th.attrs else '' for th in head.find_all('th')]
    columns_exp_tip = [th.attrs['data-tip']  if 'data-tip' in th.attrs else '' for th in head.find_all('th')]
    body = table.tbody
    data = []
    for tr in body.find_all('tr'):
        if 'class' not in tr.attrs:
            name_div = tr.find('th')
            team_name = name_div.a['title']
            temp = [td.text.strip() for td in tr.find_all('td')]
            data.append([team_name+'(' + name_div.text + ')'] + temp)
    
    df_pitching = pd.DataFrame(data, columns=columns_name)
    df_pitching_exp = pd.DataFrame([columns_exp_stat, columns_exp_tip], columns=columns_name)
    return df_pitching, df_pitching_exp

def process_fielding(content):
    fielding = content.find('div', attrs={'id':'all_teams_standard_fielding'})
    table = fielding.find('table')
    head = table.thead.find('tr')
    columns_name = [th.text.strip() for th in head.find_all('th')]
    #print(columns_name)
    #print(len(columns_name))
    columns_exp_stat = [th.attrs['data-stat'] if 'data-stat' in th.attrs else '' for th in head.find_all('th')]
    columns_exp_tip = [th.attrs['data-tip']  if 'data-tip' in th.attrs else '' for th in head.find_all('th')]
    body = table.tbody
    data = []
    for tr in body.find_all('tr'):
        if 'class' not in tr.attrs:
            name_div = tr.find('th')
            team_name = name_div.a['title']
            temp = [td.text.strip() for td in tr.find_all('td')]
            data.append([team_name+'(' + name_div.text + ')'] + temp)
    
    df_fielding = pd.DataFrame(data, columns=columns_name)
    df_fielding_exp = pd.DataFrame([columns_exp_stat, columns_exp_tip], columns=columns_name)
    return df_fielding, df_fielding_exp

def process_year(page):
    
    content = get_parsed_remove_comment(baseball_base_url + page)
    
    df_division = process_division(content)
    df_post = process_post(content)
    df_batting, df_batting_exp = process_batting(content)
    df_pitching, df_pitching_exp = process_pitching(content)
    df_fielding, df_fielding_exp = process_fielding(content)
    
    return df_division, df_post, df_batting, df_batting_exp, df_pitching, df_pitching_exp, df_fielding, df_fielding_exp
    

for idd, year in enumerate(range(min(year_league_df.shape[0], 10000))):
    year_str = year_league_df.loc[year, 'Year']
    print(year_str)
    link = year_league_df.loc[year, 'Lg_url']
    df_division, df_post, df_batting, df_batting_exp, df_pitching, df_pitching_exp, df_fielding, df_fielding_exp = process_year(link)
    obj = (df_division, df_post, df_batting, df_batting_exp, df_pitching, df_pitching_exp, df_fielding, df_fielding_exp)
    file_name = f'{idd:02}' + '_' + year_str + '.pkl'
    path = os.path.join(folder, file_name)
    with open(path, 'wb') as f:
        pickle.dump(obj, f)

2020
2020
2019
2019
2018
2018
2017
2017
2016
2016
2015
2015
2014
2014
2013
2013
2012
2012
2011
2011
2010
2010
2009
2009
2008
2008
2007
2007
2006
2006
2005
2005
2004
2004
2003
2003
2002
2002
2001
2001
2000
2000
1999
1999
1998
1998
1997
1997
1996
1996
1995
1995
1994
1994
1993
1993
1992
1992
1991
1991
1990
1990
1989
1989
1988
1988
1987
1987
1986
1986
1985
1985
1984
1984
1983
1983
1982
1982
1981
1981
1980
1980


In [24]:
path

'major_leagues\\81_1980.pkl'

In [21]:
with open(path, 'rb') as f:
    v = pickle.load(f)

In [22]:
len(v)

8

In [28]:
v[6]

,Tm,#Fld,RA/G,DefEff,G,GS,CG,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr
0,Baltimore Orioles(BAL),34,3.95,.709,162,1458,1030,13140.0,6293,4380,1818,95,178,.985,28,3
1,Boston Red Sox(BOS),41,4.79,.686,160,1440,1173,12972.0,6410,4324,1937,149,206,.977,9,1
2,California Angels(CAL),40,4.98,.690,160,1440,1126,12855.0,5995,4285,1576,134,144,.978,-64,-6
3,Chicago White Sox(CHW),38,4.46,.699,162,1458,1078,12918.0,6400,4306,1923,171,162,.973,-12,-1
4,Cleveland Indians(CLE),32,5.04,.689,160,1440,1111,12852.0,6102,4284,1713,105,143,.983,-46,-4
5,Detroit Tigers(DET),34,4.64,.704,163,1467,1133,13206.0,6322,4402,1787,133,165,.979,-20,-2
6,Kansas City Royals(KCR),36,4.28,.707,162,1458,1112,13134.0,6339,4378,1820,141,150,.978,22,2
7,Milwaukee Brewers(MIL),32,4.21,.702,162,1458,1228,13050.0,6383,4350,1886,147,189,.977,25,2
8,Minnesota Twins(MIN),36,4.50,.692,161,1449,1108,13059.0,6497,4353,1996,148,192,.977,37,3
9,New York Yankees(NYY),39,4.09,.699,162,1458,1078,13179.0,6383,4393,1852,138,160,.978,-14,-1


In [35]:
year_league_df

,Year,Lg,Champion,MVP,Lg_url,Champion_url,MVP_url
0,2020,NL,Los Angeles Dodgers,Freddie Freeman,/leagues/NL/2020.shtml,/teams/LAD/2020.shtml,/players/f/freemfr01.shtml
1,2020,AL,Tampa Bay Rays,José Abreu,/leagues/AL/2020.shtml,/teams/TBR/2020.shtml,/players/a/abreujo02.shtml
2,2019,NL,Washington Nationals,Cody Bellinger,/leagues/NL/2019.shtml,/teams/WSN/2019.shtml,/players/b/bellico01.shtml
3,2019,AL,Houston Astros,Mike Trout,/leagues/AL/2019.shtml,/teams/HOU/2019.shtml,/players/t/troutmi01.shtml
4,2018,NL,Los Angeles Dodgers,Christian Yelich,/leagues/NL/2018.shtml,/teams/LAD/2018.shtml,/players/y/yelicch01.shtml
...,...,...,...,...,...,...,...
77,1982,AL,Milwaukee Brewers,Robin Yount,/leagues/AL/1982.shtml,/teams/MIL/1982.shtml,/players/y/yountro01.shtml
78,1981,NL,Los Angeles Dodgers,Mike Schmidt,/leagues/NL/1981.shtml,/teams/LAD/1981.shtml,/players/s/schmimi01.shtml
79,1981,AL,New York Yankees,Rollie Fingers,/leagues/AL/1981.shtml,/teams/NYY/1981.shtml,/players/f/fingero01.shtml
80,1980,NL,Philadelphia Phillies,Mike Schmidt,/leagues/NL/1980.shtml,/teams/PHI/1980.shtml,/players/s/schmimi01.shtml


In [46]:
wrong_count = 4
columns = ['Question', 'Correct'] + ['Wrong_' + str(i + 1) for i in range(wrong_count)] +  ['tags']

question_list = []
import random
random.seed(1000)

for idd, year in enumerate(range(min(year_league_df.shape[0], 200))):
    year_str = year_league_df.loc[year, 'Year']
    file_name = f'{idd:02}' + '_' + year_str + '.pkl'
    path = os.path.join(folder, file_name)
    with open(path, 'rb') as f:
        df_division, df_post, df_batting, df_batting_exp, df_pitching, df_pitching_exp, df_fielding, df_fielding_exp = pickle.load(f)
        
    
    # post season
    if df_post.shape[0] > 0:
        for i in range(df_post.shape[0]):
            over = df_post.loc[i, 'Over']
            series = df_post.loc[i, 'Series']
            question = f'What is the game score of {over} ' \
                + f'({series}, Postseason) in {year_str}?'
            correct = df_post.loc[i, 'Score']
            wrongs = [f'{random.randint(0, 4)}-{random.randint(0, 4)}' for i in range(wrong_count * 3)]
            wrongs = [wrong for wrong in set(wrongs) if wrong != correct]
            wrongs = wrongs[:wrong_count]
            tags = ['Postseason']
            question_list.append([question] + [correct] + wrongs + [tags])
    
    # number of batters used
    if df_batting.shape[0] > 0:
        for i in range(df_batting.shape[0]):
            team = df_batting.loc[i, 'Tm']
            batters = df_batting.loc[i, '#Bat']
            
            question = f'How many batters are used by {team} in the season of {year_str}?'
            correct = int(batters)
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = ['batters', 'team']
            question_list.append([question] + [correct] + wrongs + [tags])
            
            question = f'How many Hits/Hits Allowed (Batting) are achieved by {team} in the season of {year_str}?'
            correct = int(df_batting.loc[i, 'H'])
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = ['batters', 'team']
            question_list.append([question] + [correct] + wrongs + [tags])
    
    
    # number of pitchers used
    if df_pitching.shape[0] > 0:
        for i in range(df_pitching.shape[0]):
            team = df_pitching.loc[i, 'Tm']
            players = df_pitching.loc[i, '#P']
            
            question = f'How many pitchers are used by {team} in the season of {year_str}?'
            correct = int(players)
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = ['pitchers', 'team']
            question_list.append([question] + [correct] + wrongs + [tags])
            
            question = f'How many Hits/Hits Allowed (Pitching) are achieved by {team} in the season of {year_str}?'
            correct = int(df_pitching.loc[i, 'H'])
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = ['pitchers', 'team']
            question_list.append([question] + [correct] + wrongs + [tags])
    
    # number of fielders used
    if df_fielding.shape[0] > 0:
        for i in range(df_fielding.shape[0]):
            team = df_fielding.loc[i, 'Tm']
            players = df_fielding.loc[i, '#Fld']
            
            question = f'How many fielders are used by {team} in the season of {year_str}?'
            correct = int(players)
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = ['fielders', 'team']
            question_list.append([question] + [correct] + wrongs + [tags])
            
            question = f'How many Assists (fielding) are achieved by {team} in the season of {year_str}?'
            correct = int(df_fielding.loc[i, 'A'])
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = ['fielders', 'team']
            question_list.append([question] + [correct] + wrongs + [tags])

In [47]:
len(question_list)

7300

In [49]:
league_each_season_questions = pd.DataFrame(question_list, columns=columns)
with open('questions_league_each_season.pkl', 'wb') as f:
    pickle.dump(league_each_season_questions, f)
with open('questions_league_each_season.pkl', 'rb') as f:
    league_each_season_questions = pickle.load(f)